In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\ML_Projects\\Violence-Reporter'

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
os.environ['MLFLOW_TRACKING_URI']=os.getenv('MLFLOW_TRACKING_URI')
os.environ['MLFLOW_TRACKING_USERNAME']=os.getenv('MLFLOW_TRACKING_USERNAME')
os.environ['MLFLOW_TRACKING_PASSWORD']=os.getenv('MLFLOW_TRACKING_PASSWORD')

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    root_dir: Path
    path_of_model: Path
    training_data: Path
    training_metrics: Path
    all_params: dict
    mlflow_uri: str

In [23]:
from violenceReporter.constants import *
from violenceReporter.utils.common import read_yaml, create_directories, save_json

In [24]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        create_directories([
            Path(self.config.evaluation.root_dir)
        ])

        eval_config = EvaluationConfig(
            root_dir=Path(self.config.evaluation.root_dir),
            path_of_model=Path(self.config.training.trained_model_path),
            training_metrics=Path(self.config.evaluation.training_metrics),
            training_data=Path(self.config.data_transformation.root_dir),
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI'],
            all_params=self.params,
        )
        return eval_config
    

In [25]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from pathlib import Path
%matplotlib inline

In [34]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config
    
    def plot_metric(self,model_training_history, metric_name_1, metric_name_2, plot_name):
        metric_value_1 = model_training_history[metric_name_1]
        metric_value_2 = model_training_history[metric_name_2]
        epochs = range(len(metric_value_1))
    
        plt.plot(epochs, metric_value_1, 'blue', label=metric_name_1)
        plt.plot(epochs, metric_value_2, 'orange', label=metric_name_2)
        plt.title(str(plot_name))
        plt.legend()


    def generate_training_metrics(self):
        model_history = pd.read_csv(self.config.training_metrics)

        # Plot loss
        self.plot_metric(model_history, 'loss', 'val_loss', 'Total Loss vs Total Validation Loss')
        plt.savefig(os.path.join(self.config.root_dir,'loss_plot.png'))
        plt.close()

        # Plot accuracy
        self.plot_metric(model_history, 'accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')
        plt.savefig(os.path.join(self.config.root_dir,'accuracy_plot.png'))
        plt.close()

        # Plot confusion matrix
        cm = confusion_matrix(self.labels_test, self.labels_predict)
        ax = plt.subplot()
        sns.heatmap(cm, annot=True, fmt='g', ax=ax)
        ax.set_xlabel('Predicted labels')
        ax.set_ylabel('True labels')
        ax.set_title('Confusion Matrix')
        ax.xaxis.set_ticklabels(['True', 'False'])
        ax.yaxis.set_ticklabels(['NonViolence', 'Violence'])
        plt.savefig(os.path.join(self.config.root_dir,'confusion_matrix.png'))
        plt.close()

        # Save classification report
        classification_report_str = classification_report(self.labels_test, self.labels_predict)
        with open(os.path.join(self.config.root_dir,'classification_report.txt'), 'w') as f:
            f.write(classification_report_str)


        
    def test_set_generator(self):
        features = np.load(os.path.join(self.config.training_data,"features.npy"))
        labels= np.load(os.path.join(self.config.training_data,"labels.npy"))
        video_files_paths = np.load(os.path.join(self.config.training_data,"video_files_paths.npy"))

        one_hot_encoded_labels = to_categorical(labels)
        features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size=0.2,
                                                                            shuffle=True, random_state=42)

        self.features_test=features_test
        self.labels_test=labels_test

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def model_predict(self):
        labels_predict = self.model.predict(self.features_test)
        self.labels_predict = np.argmax(labels_predict, axis=1)
        self.labels_test = np.argmax(self.labels_test, axis=1)
    
    def model_evaluate(self):
        self.score = self.model.evaluate(self.features_test, self.labels_test)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self.test_set_generator()
        self.model_evaluate()
        self.model_predict()
        self.save_score()
        self.generate_training_metrics()
    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="MobileNetV2")
            else:
                mlflow.keras.log_model(self.model, "model")
    

In [35]:
try:
    config=ConfigurationManager()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-04-01 18:16:53,894: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-01 18:16:53,903: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-01 18:16:53,904: INFO: common: created directory at: artifacts]
[2024-04-01 18:16:53,906: INFO: common: created directory at: artifacts\evalution]


5/5 [==============================] - 4s 524ms/step
[2024-04-01 18:17:06,168: INFO: common: json file saved at: scores.json]
[2024-04-01 18:17:06,523: WARNING: connectionpool: Retrying (Retry(total=4, connect=4, read=5, redirect=5, status=5)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026605557160>: Failed to resolve 'dagshub.com' ([Errno 11001] getaddrinfo failed)")': /JaiSehgal007/violence-reporter.mlflow/api/2.0/mlflow/runs/create]
[2024-04-01 18:17:10,533: WARNING: connectionpool: Retrying (Retry(total=3, connect=3, read=5, redirect=5, status=5)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000026605557F70>: Failed to resolve 'dagshub.com' ([Errno 11001] getaddrinfo failed)")': /JaiSehgal007/violence-reporter.mlflow/api/2.0/mlflow/runs/create]
[2024-04-01 18:17:18,543: WARNING: connectionpool: Retrying (Retry(total=2, connect=2, read=5, redirect=5, status=5)) after connec

2024/04/01 18:17:22 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-04-01 18:17:43,851: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 56). These functions will not be directly callable after loading.]
[2024-04-01 18:17:48,597: INFO: builder_impl: Assets written to: C:\Users\DELL\AppData\Local\Temp\tmpxu1feakn\model\data\model\assets]


e:\Anaconda\envs\reporter\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2024-04-01 18:22:50,897: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /JaiSehgal007/violence-reporter.mlflow/api/2.0/mlflow-artifacts/artifacts/5e85a1f80c42493eb7236c7976568527/84543a0851cc44f4b465ffcc2c426ab0/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2024-04-01 18:22:54,933: WARNING: connectionpool: Retrying (Retry(total=3, connect=4, read=4, redirect=5, status=5)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x00000265FEE685B0>: Failed to resolve 'dagshub.com' ([Errno 11001] getaddrinfo failed)")': /JaiSehgal007/violence-reporter.mlflow/api/2.0/mlflow-artifacts/artifacts/5e85a1f80c42493eb7236c7976568527/84543a0851cc44f4b465ffcc2c426ab0/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2024-04-01 18:23:0

Successfully registered model 'VGG16Model'.
2024/04/01 18:27:50 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
